In [ ]:
import os

import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

import scanpy as sc

import statsmodels.stats.multitest

In [ ]:
def count_zero_pairs(contact_mtx):
    n_0 = 0
    for i in range(contact_mtx.shape[0]):
        for j in range(i, contact_mtx.shape[0]):
            if contact_mtx[i, j] == 0:
                n_0 += 1
    return n_0

def adjust_p_value_matrix_by_BH(p_val_mtx):
    '''Adjust the p-values in a matrix by the Benjamini/Hochberg method.
    The matrix should be symmetric.
    '''
    p_val_sequential = []
    N = p_val_mtx.shape[0]
    
    for i in range(N):
        for j in range(i, N):
            p_val_sequential.append(p_val_mtx[i, j])

    p_val_sequential_bh = statsmodels.stats.multitest.multipletests(p_val_sequential, method='fdr_bh')[1]
    
    adjusted_p_val_mtx = np.zeros((N, N))
    
    counter = 0
    for i in range(N):
        for j in range(i, N):
            adjusted_p_val_mtx[i, j] = p_val_sequential_bh[counter]
            adjusted_p_val_mtx[j, i] = p_val_sequential_bh[counter]
            counter += 1
            
    return adjusted_p_val_mtx

def get_data_frame_from_metrices(cell_types, mtx_dict):
    N = len(cell_types)
    
    serials_dict = {'cell_type1':[], 'cell_type2':[]}
    for k in mtx_dict.keys():
        serials_dict[k] = []
        
    for i in range(N):
        for j in range(i, N):
            serials_dict['cell_type1'].append(cell_types[i])
            serials_dict['cell_type2'].append(cell_types[j])
            for k in mtx_dict.keys():
                serials_dict[k].append(mtx_dict[k][i, j])
                
    return pd.DataFrame(serials_dict)
    

def sort_cell_type_contact_p_values(p_val_mtx, cell_types):
    '''Return a list of (cell_type1, cell_type2, p_value) sorted by p_values.'''
    p_val_list = []
    N = p_val_mtx.shape[0]
    for i in range(N):
        for j in range(i, N):
            p_val_list.append((cell_types[i], cell_types[j], p_val_mtx[i, j]))
    return sorted(p_val_list, key=lambda x:x[2])

In [ ]:
import scipy.cluster
from scattermap import scattermap

def get_optimal_order_of_mtx(X):
    Z = scipy.cluster.hierarchy.ward(X)
    return scipy.cluster.hierarchy.leaves_list(
        scipy.cluster.hierarchy.optimal_leaf_ordering(Z, X))

def get_ordered_tick_labels(tick_labels):
    tick_labels_with_class = [s.split(' ')[-1] + ' ' + s for s in tick_labels]
    return np.argsort(tick_labels_with_class)

def filter_pval_mtx(pval_mtx, tick_labels, allowed_pairs):
    pval_mtx_filtered = pval_mtx.copy()
    
    for i in range(pval_mtx.shape[0]):
        ct1 = tick_labels[i]
        for j in range(pval_mtx.shape[1]):
            ct2 = tick_labels[j]
            
            if ((ct1, ct2) in allowed_pairs) or ((ct2, ct1) in allowed_pairs):
                continue
            else:
                pval_mtx_filtered[i, j] = 1
            
    return pval_mtx_filtered

def make_dotplot(pval_mtx, fold_change_mtx, tick_labels, title='', allowed_pairs=None):

    #optimal_order = get_optimal_order_of_mtx(pval_mtx)
    optimal_order = get_ordered_tick_labels(tick_labels)
    
    pval_mtx = pval_mtx[optimal_order][:, optimal_order]
    fold_change_mtx = fold_change_mtx[optimal_order][:, optimal_order]
    tick_labels = tick_labels[optimal_order]
    
    if None is not allowed_pairs:
        pval_mtx = filter_pval_mtx(pval_mtx, tick_labels, allowed_pairs)
    
    mlog_pvals = - np.log10(np.maximum(pval_mtx, 1e-10))

    fig_len = len(tick_labels) * 0.1
    fig = plt.figure(figsize=(fig_len, fig_len), dpi=300)


    ax = scattermap(fold_change_mtx, marker_size=mlog_pvals + 0.5, 
                square=True, 
                cmap="coolwarm",
                linewidths=0.2 * (pval_mtx < 0.05).reshape(-1), 
                linecolor='black', xticklabels=tick_labels, yticklabels=tick_labels,
                vmin=0, vmax=2, 
                cbar_kws={'shrink':0.5, 'anchor':(0, 0.7)})
    ax.tick_params(axis='both', which='major', labelsize=4.5)
    ax.figure.axes[1].tick_params(axis="y", labelsize=7)
    ax.figure.axes[1].set_ylabel('fold change', fontsize=7)

    # Create a dot size legend using off-axis scatter calls and legend
    ax.scatter(-1, -1, label='$10^{-10}$', marker="o", linewidths=0, c="grey", s=10.5)
    ax.scatter(-1, -1, label='$10^{-2}$', marker="o", linewidths=0, c="grey", s=2.5)
    ax.scatter(-1, -1, label='1', marker="o", linewidths=0, c="grey", s=0.5)
    leg = ax.legend(loc="upper left", bbox_to_anchor=(1, 0.2), fontsize=7)
    leg.set_title('adjusted p-val',prop={'size':7})
    
    ax.set_title(title)
    
    return ax

In [ ]:
import json
with open('/home/xingjiepan/data/whole_brain/analysis/20230808_ligand_receptor_analysis/contact_enriched_lr_pairs/allowed_pairs_by_regions.json') as f:
    allowed_pairs_by_regions = json.load(f)
    
for r in allowed_pairs_by_regions:
    allowed_pairs_by_regions[r] = [tuple(p) for p in allowed_pairs_by_regions[r]]

In [ ]:
%%time

permutation_path = 'outputs_30um'

major_brain_regions = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']

result_dfs = []

for region in major_brain_regions:
    
    # Load the cell type labels
    df_ct_labels = pd.read_csv(os.path.join('cells_by_regions', f'{region}.csv'), index_col=0)

    subclass_types = np.unique(df_ct_labels['subclass_label_transfer'])
    
    cell_contact_counts = np.load(os.path.join(permutation_path, f'{region}_no_permutation.npy'))

    local_null_means = np.load(os.path.join(permutation_path, f'{region}_local_permutation_mean.npy'))
    local_null_stds = np.load(os.path.join(permutation_path, f'{region}_local_permutation_std.npy'))

    # Require all stds to be larger or equal to the minimal observable std value
    local_null_stds = np.maximum(local_null_stds, np.sqrt(1 / 1000))
    
    local_z_scores = (cell_contact_counts - local_null_means) / local_null_stds
    local_p_values = scipy.stats.norm.sf(local_z_scores)
    adjusted_local_p_values = adjust_p_value_matrix_by_BH(local_p_values)
    
    fold_changes = cell_contact_counts / (local_null_means + 1e-4)
    
    #make_dotplot(local_p_values, fold_changes, subclass_types, title=region)
    #make_dotplot(adjusted_local_p_values, fold_changes, subclass_types, title=region)
    make_dotplot(adjusted_local_p_values, fold_changes, subclass_types, title=region, 
                 allowed_pairs=allowed_pairs_by_regions[region])
    
    # Gather all results into a data frame
    contact_result_df = get_data_frame_from_metrices(subclass_types, 
                                             {'pval-adjusted': adjusted_local_p_values,
                                              'pval': local_p_values,
                                              'z_score': local_z_scores,
                                              'contact_count': cell_contact_counts,
                                              'permutation_mean': local_null_means,
                                              'permutation_std': local_null_stds
                                            }).sort_values('z_score', ascending=False)

    # Filter out pairs that don't contact
    contact_result_df = contact_result_df[contact_result_df['pval-adjusted'] < 0.05]
    contact_result_df = contact_result_df[contact_result_df['contact_count'] > 50]
    contact_result_df.to_csv(os.path.join(permutation_path, f'{region}_close_contacts.csv'))
    
    
    result_dfs.append(contact_result_df)
    #break

In [ ]:
combined_results = pd.concat(result_dfs)
combined_results

In [ ]:
#ct1, ct2 = 'BAM NN', 'Endo NN'
#ct1, ct2 = 'SCs Dmbx1 Gaba', 'SCig Foxb1 Glut'
#ct1, ct2 = 'CHOR NN', 'Astro-Epen NN'
ct1, ct2 = 'Vip Gaba', 'L2/3 IT CTX Glut'


combined_results[((combined_results['cell_type1'] == ct1) & (combined_results['cell_type2'] == ct2))
                |((combined_results['cell_type1'] == ct2) & (combined_results['cell_type2'] == ct1))]

In [ ]:
all_pairs = combined_results[['cell_type1', 'cell_type2']].values
selected_pairs = []
selected_ids = []

for i in range(combined_results.shape[0]):
    p =  tuple(sorted(all_pairs[i]))
    
    if p not in selected_pairs:
        selected_pairs.append(p)
        selected_ids.append(i)
        
#combined_results.iloc[selected_ids].to_csv(os.path.join(permutation_path,
#    'subclass_close_contacts_non_redundant.csv'))
combined_results.iloc[selected_ids]

In [ ]:
pair_set_15um = {('ABC NN', 'ABC NN'),
 ('ABC NN', 'Astro-TE NN'),
 ('ABC NN', 'BAM NN'),
 ('ABC NN', 'Endo NN'),
 ('ABC NN', 'OEC NN'),
 ('ABC NN', 'Peri NN'),
 ('ABC NN', 'SMC NN'),
 ('ABC NN', 'VLMC NN'),
 ('ACB-BST-FS D1 Gaba', 'ACB-BST-FS D1 Gaba'),
 ('AD Serpinb7 Glut', 'AD Serpinb7 Glut'),
 ('ADP-MPO Trp73 Glut', 'ADP-MPO Trp73 Glut'),
 ('AHN Onecut3 Gaba', 'AHN Onecut3 Gaba'),
 ('APN C1ql2 Glut', 'APN C1ql2 Glut'),
 ('APN C1ql4 Glut', 'APN C1ql4 Glut'),
 ('ARH-PVp Tbx3 Gaba', 'ARH-PVp Tbx3 Gaba'),
 ('ARH-PVp Tbx3 Glut', 'ARH-PVp Tbx3 Glut'),
 ('AVPV-MEPO-SFO Tbr1 Glut', 'AVPV-MEPO-SFO Tbr1 Glut'),
 ('AVPV-MEPO-SFO Tbr1 Glut', 'Tanycyte NN'),
 ('Astro-CB NN', 'Astro-CB NN'),
 ('Astro-CB NN', 'Bergmann NN'),
 ('Astro-CB NN', 'CB Granule Glut'),
 ('Astro-CB NN', 'Endo NN'),
 ('Astro-CB NN', 'Microglia NN'),
 ('Astro-CB NN', 'Oligo NN'),
 ('Astro-NT NN', 'Astro-NT NN'),
 ('Astro-NT NN', 'BAM NN'),
 ('Astro-NT NN', 'Endo NN'),
 ('Astro-NT NN', 'Ependymal NN'),
 ('Astro-NT NN', 'Microglia NN'),
 ('Astro-NT NN', 'OPC NN'),
 ('Astro-NT NN', 'Oligo NN'),
 ('Astro-NT NN', 'Peri NN'),
 ('Astro-NT NN', 'SCsg Gabrr2 Gaba'),
 ('Astro-NT NN', 'SMC NN'),
 ('Astro-NT NN', 'TRS-BAC Sln Glut'),
 ('Astro-NT NN', 'VLMC NN'),
 ('Astro-OLF NN', 'Astro-OLF NN'),
 ('Astro-OLF NN', 'Endo NN'),
 ('Astro-OLF NN', 'Microglia NN'),
 ('Astro-OLF NN', 'OB Dopa-Gaba'),
 ('Astro-OLF NN', 'OB Eomes Ms4a15 Glut'),
 ('Astro-OLF NN', 'OB Meis2 Thsd7b Gaba'),
 ('Astro-OLF NN', 'OB-STR-CTX Inh IMN'),
 ('Astro-OLF NN', 'OB-out Frmd7 Gaba'),
 ('Astro-OLF NN', 'Oligo NN'),
 ('Astro-OLF NN', 'Peri NN'),
 ('Astro-OLF NN', 'SMC NN'),
 ('Astro-TE NN', 'Astro-TE NN'),
 ('Astro-TE NN', 'BAM NN'),
 ('Astro-TE NN', 'DG-PIR Ex IMN'),
 ('Astro-TE NN', 'Endo NN'),
 ('Astro-TE NN', 'Ependymal NN'),
 ('Astro-TE NN', 'HPF CR Glut'),
 ('Astro-TE NN', 'Microglia NN'),
 ('Astro-TE NN', 'OB-STR-CTX Inh IMN'),
 ('Astro-TE NN', 'OPC NN'),
 ('Astro-TE NN', 'Oligo NN'),
 ('Astro-TE NN', 'Peri NN'),
 ('Astro-TE NN', 'SMC NN'),
 ('Astro-TE NN', 'VLMC NN'),
 ('Astroependymal NN', 'Astroependymal NN'),
 ('Astroependymal NN', 'Ependymal NN'),
 ('Astroependymal NN', 'MH Tac2 Glut'),
 ('Astroependymal NN', 'VLMC NN'),
 ('BAM NN', 'BAM NN'),
 ('BAM NN', 'Bergmann NN'),
 ('BAM NN', 'CHOR NN'),
 ('BAM NN', 'Endo NN'),
 ('BAM NN', 'Microglia NN'),
 ('BAM NN', 'OEC NN'),
 ('BAM NN', 'Peri NN'),
 ('BAM NN', 'SMC NN'),
 ('BAM NN', 'VLMC NN'),
 ('BST Tac2 Gaba', 'BST Tac2 Gaba'),
 ('BST-MPN Six3 Nrgn Gaba', 'BST-MPN Six3 Nrgn Gaba'),
 ('BST-po Iigp1 Glut', 'BST-po Iigp1 Glut'),
 ('Bergmann NN', 'Bergmann NN'),
 ('Bergmann NN', 'CB PLI Gly-Gaba'),
 ('Bergmann NN', 'CBX Golgi Gly-Gaba'),
 ('Bergmann NN', 'CBX Purkinje Gaba'),
 ('Bergmann NN', 'Endo NN'),
 ('Bergmann NN', 'Oligo NN'),
 ('Bergmann NN', 'Peri NN'),
 ('CA1-ProS Glut', 'CA1-ProS Glut'),
 ('CA1-ProS Glut', 'Pvalb chandelier Gaba'),
 ('CA2-FC-IG Glut', 'CA2-FC-IG Glut'),
 ('CA3 Glut', 'CA3 Glut'),
 ('CB Granule Glut', 'CB Granule Glut'),
 ('CB Granule Glut', 'CB PLI Gly-Gaba'),
 ('CB Granule Glut', 'CBX Golgi Gly-Gaba'),
 ('CB Granule Glut', 'DCO UBC Glut'),
 ('CB PLI Gly-Gaba', 'CB PLI Gly-Gaba'),
 ('CB PLI Gly-Gaba', 'CBX Purkinje Gaba'),
 ('CBN Dmbx1 Gaba', 'CBN Dmbx1 Gaba'),
 ('CBN Dmbx1 Gaba', 'Oligo NN'),
 ('CBX MLI Cdh22 Gaba', 'CBX MLI Cdh22 Gaba'),
 ('CBX MLI Cdh22 Gaba', 'CBX MLI Megf11 Gaba'),
 ('CEA-AAA-BST Six3 Sp9 Gaba', 'CEA-AAA-BST Six3 Sp9 Gaba'),
 ('CEA-BST Ebf1 Pdyn Gaba', 'CEA-BST Ebf1 Pdyn Gaba'),
 ('CHOR NN', 'CHOR NN'),
 ('CHOR NN', 'Endo NN'),
 ('CHOR NN', 'Lymphoid NN'),
 ('CHOR NN', 'Peri NN'),
 ('CHOR NN', 'VLMC NN'),
 ('CLA-EPd-CTX Car3 Glut', 'CLA-EPd-CTX Car3 Glut'),
 ('CLA-EPd-CTX Car3 Glut', 'L2/3 IT RSP Glut'),
 ('CM-IAD-CL-PCN Sema5b Glut', 'CM-IAD-CL-PCN Sema5b Glut'),
 ('COAa-PAA-MEA Barhl2 Glut', 'COAa-PAA-MEA Barhl2 Glut'),
 ('COAa-PAA-MEA Barhl2 Glut', 'Oligo NN'),
 ('COAp Grxcr2 Glut', 'COAp Grxcr2 Glut'),
 ('CS-PRNr-PCG Tmem163 Otp Gaba', 'CS-PRNr-PCG Tmem163 Otp Gaba'),
 ('CS-RPO Meis2 Gaba', 'CS-RPO Meis2 Gaba'),
 ('CT SUB Glut', 'CT SUB Glut'),
 ('CT SUB Glut', 'VLMC NN'),
 ('CUN-PPN Evx2 Meis2 Glut', 'CUN-PPN Evx2 Meis2 Glut'),
 ('DC NN', 'Endo NN'),
 ('DC NN', 'VLMC NN'),
 ('DCO UBC Glut', 'DCO UBC Glut'),
 ('DG Glut', 'DG Glut'),
 ('DG-PIR Ex IMN', 'DG-PIR Ex IMN'),
 ('DG-PIR Ex IMN', 'Endo NN'),
 ('DG-PIR Ex IMN', 'Lamp5 Lhx6 Gaba'),
 ('DG-PIR Ex IMN', 'Microglia NN'),
 ('DG-PIR Ex IMN', 'Oligo NN'),
 ('DMH Hmx2 Gaba', 'DMH Hmx2 Gaba'),
 ('DMH Hmx2 Glut', 'DMH Hmx2 Glut'),
 ('DMH Nkx2-4 Glut', 'DMH Nkx2-4 Glut'),
 ('DMH-LHA Gsx1 Gaba', 'DMH-LHA Gsx1 Gaba'),
 ('DTN-LDT-IPN Otp Pax3 Gaba', 'DTN-LDT-IPN Otp Pax3 Gaba'),
 ('ENTmv-PA-COAp Glut', 'ENTmv-PA-COAp Glut'),
 ('Endo NN', 'Endo NN'),
 ('Endo NN', 'Lymphoid NN'),
 ('Endo NN', 'MH Tac2 Glut'),
 ('Endo NN', 'Microglia NN'),
 ('Endo NN', 'OB-STR-CTX Inh IMN'),
 ('Endo NN', 'OB-out Frmd7 Gaba'),
 ('Endo NN', 'OEC NN'),
 ('Endo NN', 'OPC NN'),
 ('Endo NN', 'Oligo NN'),
 ('Endo NN', 'PVH-SO-PVa Otp Glut'),
 ('Endo NN', 'Peri NN'),
 ('Endo NN', 'SMC NN'),
 ('Endo NN', 'TRS-BAC Sln Glut'),
 ('Endo NN', 'VLMC NN'),
 ('Ependymal NN', 'Ependymal NN'),
 ('Ependymal NN', 'MEA-BST Sox6 Gaba'),
 ('Ependymal NN', 'Microglia NN'),
 ('Ependymal NN', 'OB-STR-CTX Inh IMN'),
 ('Ependymal NN', 'Tanycyte NN'),
 ('GPe-SI Sox6 Cyp26b1 Gaba', 'GPe-SI Sox6 Cyp26b1 Gaba'),
 ('GPe-SI Sox6 Cyp26b1 Gaba', 'Oligo NN'),
 ('HB Calcb Chol', 'HB Calcb Chol'),
 ('HPF CR Glut', 'HPF CR Glut'),
 ('Hypendymal NN', 'Hypendymal NN'),
 ('IC Six3 En2 Gaba', 'IC Six3 En2 Gaba'),
 ('IF-RL-CLI-PAG Foxa1 Glut', 'IF-RL-CLI-PAG Foxa1 Glut'),
 ('IPN Otp Crisp1 Gaba', 'IPN Otp Crisp1 Gaba'),
 ('IPN-LDT Vsx2 Nkx6-1 Glut', 'IPN-LDT Vsx2 Nkx6-1 Glut'),
 ('IT AON-TT-DP Glut', 'IT AON-TT-DP Glut'),
 ('IT EP-CLA Glut', 'IT EP-CLA Glut'),
 ('L2 IT PPP-APr Glut', 'L2 IT PPP-APr Glut'),
 ('L2/3 IT CTX Glut', 'L2/3 IT CTX Glut'),
 ('L2/3 IT CTX Glut', 'L2/3 IT RSP Glut'),
 ('L2/3 IT CTX Glut', 'Vip Gaba'),
 ('L2/3 IT PIR-ENTl Glut', 'L2/3 IT PIR-ENTl Glut'),
 ('L2/3 IT PPP Glut', 'L2/3 IT PPP Glut'),
 ('L2/3 IT RSP Glut', 'L2/3 IT RSP Glut'),
 ('L2/3 IT RSP Glut', 'Vip Gaba'),
 ('L5 IT CTX Glut', 'L5 IT CTX Glut'),
 ('L5 NP CTX Glut', 'L5 NP CTX Glut'),
 ('L5 PPP Glut', 'L5 PPP Glut'),
 ('L5 PPP Glut', 'NP PPP Glut'),
 ('L5/6 IT TPE-ENT Glut', 'L5/6 IT TPE-ENT Glut'),
 ('L6 CT CTX Glut', 'L6 CT CTX Glut'),
 ('L6 IT CTX Glut', 'L6 IT CTX Glut'),
 ('L6b CTX Glut', 'L6b CTX Glut'),
 ('L6b/CT ENT Glut', 'L6b/CT ENT Glut'),
 ('LDT-PCG St18 Gaba', 'LDT-PCG St18 Gaba'),
 ('LDT-PCG Vsx2 Lhx4 Glut', 'LDT-PCG Vsx2 Lhx4 Glut'),
 ('LDT-PCG-CS Gata3 Lhx1 Gaba', 'LDT-PCG-CS Gata3 Lhx1 Gaba'),
 ('LGv-SPFp-SPFm Nkx2-2 Tcf7l2 Gaba', 'LGv-SPFp-SPFm Nkx2-2 Tcf7l2 Gaba'),
 ('LH Pou4f1 Sox1 Glut', 'LH Pou4f1 Sox1 Glut'),
 ('LHA-AHN-PVH Otp Trh Glut', 'LHA-AHN-PVH Otp Trh Glut'),
 ('LSX Nkx2-1 Gaba', 'LSX Nkx2-1 Gaba'),
 ('LSX Otx2 Gaba', 'LSX Otx2 Gaba'),
 ('LSX Prdm12 Zeb2 Gaba', 'LSX Prdm12 Zeb2 Gaba'),
 ('LSX Sall3 Lmo1 Gaba', 'LSX Sall3 Lmo1 Gaba'),
 ('LSX Sall3 Pax6 Gaba', 'LSX Sall3 Pax6 Gaba'),
 ('Lamp5 Gaba', 'Lamp5 Gaba'),
 ('Lamp5 Lhx6 Gaba', 'Lamp5 Lhx6 Gaba'),
 ('Lymphoid NN', 'Lymphoid NN'),
 ('Lymphoid NN', 'OEC NN'),
 ('Lymphoid NN', 'Peri NN'),
 ('Lymphoid NN', 'VLMC NN'),
 ('MB-MY Tph2 Glut-Sero', 'MB-MY Tph2 Glut-Sero'),
 ('MEA-BST Lhx6 Nfib Gaba', 'MEA-BST Lhx6 Nfib Gaba'),
 ('MEA-BST Lhx6 Sp9 Gaba', 'MEA-BST Lhx6 Sp9 Gaba'),
 ('MEA-BST Sox6 Gaba', 'MEA-BST Sox6 Gaba'),
 ('MEA-COA-BMA Ccdc42 Glut', 'MEA-COA-BMA Ccdc42 Glut'),
 ('MH Tac2 Glut', 'MH Tac2 Glut'),
 ('MH Tac2 Glut', 'OPC NN'),
 ('MH Tac2 Glut', 'Oligo NN'),
 ('MM Foxb1 Glut', 'MM Foxb1 Glut'),
 ('MM-ant Foxb1 Glut', 'MM-ant Foxb1 Glut'),
 ('MPN-MPO-PVpo Hmx2 Glut', 'MPN-MPO-PVpo Hmx2 Glut'),
 ('MPO-ADP Lhx8 Gaba', 'MPO-ADP Lhx8 Gaba'),
 ('MRN-VTN-PPN Pax5 Cdh23 Gaba', 'MRN-VTN-PPN Pax5 Cdh23 Gaba'),
 ('MS-SF Bsx Glut', 'MS-SF Bsx Glut'),
 ('MV-SPIV-PRP Dmbx1 Gly-Gaba', 'MV-SPIV-PRP Dmbx1 Gly-Gaba'),
 ('Microglia NN', 'Microglia NN'),
 ('Microglia NN', 'OB Dopa-Gaba'),
 ('Microglia NN', 'OB Meis2 Thsd7b Gaba'),
 ('Microglia NN', 'OB-STR-CTX Inh IMN'),
 ('Microglia NN', 'OB-out Frmd7 Gaba'),
 ('Microglia NN', 'OEC NN'),
 ('Microglia NN', 'OPC NN'),
 ('Microglia NN', 'Oligo NN'),
 ('Microglia NN', 'Peri NN'),
 ('Microglia NN', 'SMC NN'),
 ('Microglia NN', 'VLMC NN'),
 ('ND-INC Foxd2 Glut', 'ND-INC Foxd2 Glut'),
 ('NDB-SI-MA-STRv Lhx8 Gaba', 'NDB-SI-MA-STRv Lhx8 Gaba'),
 ('NDB-SI-ant Prdm12 Gaba', 'NDB-SI-ant Prdm12 Gaba'),
 ('NLL Gata3 Gly-Gaba', 'NLL Gata3 Gly-Gaba'),
 ('NP PPP Glut', 'NP PPP Glut'),
 ('NP SUB Glut', 'NP SUB Glut'),
 ('NTS Dbh Glut', 'NTS Dbh Glut'),
 ('NTS Phox2b Glut', 'NTS Phox2b Glut'),
 ('NTS-PARN Neurod2 Gly-Gaba', 'NTS-PARN Neurod2 Gly-Gaba'),
 ('OB Dopa-Gaba', 'OB Dopa-Gaba'),
 ('OB Dopa-Gaba', 'OB Eomes Ms4a15 Glut'),
 ('OB Dopa-Gaba', 'OB Meis2 Thsd7b Gaba'),
 ('OB Dopa-Gaba', 'OB-STR-CTX Inh IMN'),
 ('OB Dopa-Gaba', 'OB-out Frmd7 Gaba'),
 ('OB Dopa-Gaba', 'Oligo NN'),
 ('OB Dopa-Gaba', 'Peri NN'),
 ('OB Eomes Ms4a15 Glut', 'OB Eomes Ms4a15 Glut'),
 ('OB Eomes Ms4a15 Glut', 'OB Meis2 Thsd7b Gaba'),
 ('OB Eomes Ms4a15 Glut', 'OB-out Frmd7 Gaba'),
 ('OB Meis2 Thsd7b Gaba', 'OB Meis2 Thsd7b Gaba'),
 ('OB Meis2 Thsd7b Gaba', 'OB-STR-CTX Inh IMN'),
 ('OB Meis2 Thsd7b Gaba', 'OB-out Frmd7 Gaba'),
 ('OB Meis2 Thsd7b Gaba', 'OPC NN'),
 ('OB Trdn Gaba', 'OB Trdn Gaba'),
 ('OB Trdn Gaba', 'OB-in Frmd7 Gaba'),
 ('OB-STR-CTX Inh IMN', 'OB-STR-CTX Inh IMN'),
 ('OB-STR-CTX Inh IMN', 'OB-out Frmd7 Gaba'),
 ('OB-STR-CTX Inh IMN', 'OPC NN'),
 ('OB-STR-CTX Inh IMN', 'Oligo NN'),
 ('OB-STR-CTX Inh IMN', 'Peri NN'),
 ('OB-in Frmd7 Gaba', 'OB-in Frmd7 Gaba'),
 ('OB-mi Frmd7 Gaba', 'OB-mi Frmd7 Gaba'),
 ('OB-out Frmd7 Gaba', 'OB-out Frmd7 Gaba'),
 ('OB-out Frmd7 Gaba', 'OPC NN'),
 ('OB-out Frmd7 Gaba', 'Oligo NN'),
 ('OB-out Frmd7 Gaba', 'Peri NN'),
 ('OEC NN', 'OEC NN'),
 ('OEC NN', 'OPC NN'),
 ('OEC NN', 'Oligo NN'),
 ('OEC NN', 'Peri NN'),
 ('OEC NN', 'VLMC NN'),
 ('OPC NN', 'OPC NN'),
 ('OPC NN', 'Oligo NN'),
 ('OPC NN', 'Peri NN'),
 ('OPC NN', 'STR Prox1 Lhx6 Gaba'),
 ('OPC NN', 'Sst Gaba'),
 ('OT D3 Folh1 Gaba', 'OT D3 Folh1 Gaba'),
 ('OT D3 Folh1 Gaba', 'SI-MA-LPO-LHA Skor1 Glut'),
 ('Oligo NN', 'Oligo NN'),
 ('Oligo NN', 'Peri NN'),
 ('Oligo NN', 'RT-ZI Gnb3 Gaba'),
 ('Oligo NN', 'SI-MA-LPO-LHA Skor1 Glut'),
 ('Oligo NN', 'STR-PAL Chst9 Gaba'),
 ('Oligo NN', 'Sst Chodl Gaba'),
 ('Oligo NN', 'TRS-BAC Sln Glut'),
 ('PAG Pou4f1 Ebf2 Glut', 'PAG Pou4f1 Ebf2 Glut'),
 ('PAG Pou4f2 Glut', 'PAG Pou4f2 Glut'),
 ('PAG Pou4f2 Mesi2 Glut', 'PAG Pou4f2 Mesi2 Glut'),
 ('PAG-MRN Pou3f1 Glut', 'PAG-MRN Pou3f1 Glut'),
 ('PAG-MRN-RN Foxa2 Gaba', 'PAG-MRN-RN Foxa2 Gaba'),
 ('PAG-ND-PCG Onecut1 Gaba', 'PAG-ND-PCG Onecut1 Gaba'),
 ('PAG-PPN Pax5 Sox21 Gaba', 'PAG-PPN Pax5 Sox21 Gaba'),
 ('PAG-SC Pou4f1 Zic1 Glut', 'PAG-SC Pou4f1 Zic1 Glut'),
 ('PAL-STR Gaba-Chol', 'PAL-STR Gaba-Chol'),
 ('PARN-MDRNd-NTS Gbx2 Gly-Gaba', 'PARN-MDRNd-NTS Gbx2 Gly-Gaba'),
 ('PAS-MV Ebf2 Gly-Gaba', 'PAS-MV Ebf2 Gly-Gaba'),
 ('PB Evx2 Glut', 'PB Evx2 Glut'),
 ('PB Lmx1a Glut', 'PB Lmx1a Glut'),
 ('PB-NTS Phox2b Ebf3 Lmx1b Glut', 'PB-NTS Phox2b Ebf3 Lmx1b Glut'),
 ('PBG Mtnr1a Glut-Chol', 'PBG Mtnr1a Glut-Chol'),
 ('PCG-PRNr Vsx2 Nkx6-1 Glut', 'PCG-PRNr Vsx2 Nkx6-1 Glut'),
 ('PDTg Otp Olig3 Gaba', 'PDTg Otp Olig3 Gaba'),
 ('PH-LHA Foxb1 Glut', 'PH-LHA Foxb1 Glut'),
 ('PH-SUM Foxa1 Glut', 'PH-SUM Foxa1 Glut'),
 ('PH-ant-LHA Otp Bsx Glut', 'PH-ant-LHA Otp Bsx Glut'),
 ('PMd-LHA Foxb1 Glut', 'PMd-LHA Foxb1 Glut'),
 ('PPN-CUN-PCG Otp En1 Gaba', 'PPN-CUN-PCG Otp En1 Gaba'),
 ('PRNc Otp Gly-Gaba', 'PRNc Otp Gly-Gaba'),
 ('PRNc-NI-SG-RPO Vsx2 Nr4a2 Glut', 'PRNc-NI-SG-RPO Vsx2 Nr4a2 Glut'),
 ('PRP-NI-PRNc-GRN Otp Glut', 'PRP-NI-PRNc-GRN Otp Glut'),
 ('PRT Tcf7l2 Gaba', 'PRT Tcf7l2 Gaba'),
 ('PVH-SO-PVa Otp Glut', 'PVH-SO-PVa Otp Glut'),
 ('PVHd-DMH Lhx6 Gaba', 'PVHd-DMH Lhx6 Gaba'),
 ('PVR Six3 Sox3 Gaba', 'PVR Six3 Sox3 Gaba'),
 ('PVT-PT Ntrk1 Glut', 'PVT-PT Ntrk1 Glut'),
 ('PVpo-VMPO-MPN Hmx2 Gaba', 'PVpo-VMPO-MPN Hmx2 Gaba'),
 ('Peri NN', 'Peri NN'),
 ('Peri NN', 'SMC NN'),
 ('Peri NN', 'VLMC NN'),
 ('Pineal Crx Glut', 'Pineal Crx Glut'),
 ('Pvalb Gaba', 'Pvalb Gaba'),
 ('RHP-COA Ndnf Gaba', 'RHP-COA Ndnf Gaba'),
 ('RPA Pax6 Hoxb5 Gly-Gaba', 'RPA Pax6 Hoxb5 Gly-Gaba'),
 ('RT-ZI Gnb3 Gaba', 'RT-ZI Gnb3 Gaba'),
 ('SBPV-PVa Six6 Satb2 Gaba', 'SBPV-PVa Six6 Satb2 Gaba'),
 ('SC Lef1 Otx2 Gaba', 'SC Lef1 Otx2 Gaba'),
 ('SC Lef1 Otx2 Gaba', 'SCsg Pde5a Glut'),
 ('SC Tnnt1 Gli3 Gaba', 'SC Tnnt1 Gli3 Gaba'),
 ('SCH Six6 Cdc14a Gaba', 'SCH Six6 Cdc14a Gaba'),
 ('SCig Foxb1 Glut', 'SCig Foxb1 Glut'),
 ('SCig Tfap2b Chrnb3 Glut', 'SCig Tfap2b Chrnb3 Glut'),
 ('SCiw Pitx2 Glut', 'SCiw Pitx2 Glut'),
 ('SCop Sln Glut', 'SCop Sln Glut'),
 ('SCs Dmbx1 Gaba', 'SCs Dmbx1 Gaba'),
 ('SCs Pax7 Nfia Gaba', 'SCs Pax7 Nfia Gaba'),
 ('SCsg Gabrr2 Gaba', 'SCsg Gabrr2 Gaba'),
 ('SCsg Gabrr2 Gaba', 'SCsg Pde5a Glut'),
 ('SCsg Pde5a Glut', 'SCsg Pde5a Glut'),
 ('SCsg Pde5a Glut', 'VLMC NN'),
 ('SI-MA-LPO-LHA Skor1 Glut', 'SI-MA-LPO-LHA Skor1 Glut'),
 ('SI-MPO-LPO Lhx8 Gaba', 'SI-MPO-LPO Lhx8 Gaba'),
 ('SMC NN', 'SMC NN'),
 ('SMC NN', 'VLMC NN'),
 ('SNc-VTA-RAmb Foxa1 Dopa', 'SNc-VTA-RAmb Foxa1 Dopa'),
 ('SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut', 'SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut'),
 ('SPVC Ccdc172 Glut', 'SPVC Ccdc172 Glut'),
 ('SPVI-SPVC Sall3 Lhx1 Gly-Gaba', 'SPVI-SPVC Sall3 Lhx1 Gly-Gaba'),
 ('STN-PSTN Pitx2 Glut', 'STN-PSTN Pitx2 Glut'),
 ('STR D1 Gaba', 'STR D1 Gaba'),
 ('STR D1 Sema5a Gaba', 'STR D1 Sema5a Gaba'),
 ('STR D1 Sema5a Gaba', 'STR-PAL Chst9 Gaba'),
 ('STR D2 Gaba', 'STR D2 Gaba'),
 ('STR Lhx8 Gaba', 'STR Lhx8 Gaba'),
 ('STR Prox1 Lhx6 Gaba', 'STR Prox1 Lhx6 Gaba'),
 ('STR-PAL Chst9 Gaba', 'STR-PAL Chst9 Gaba'),
 ('Sncg Gaba', 'Sncg Gaba'),
 ('Sst Chodl Gaba', 'Sst Chodl Gaba'),
 ('Sst Gaba', 'Sst Gaba'),
 ('TMd-DMH Foxd2 Gaba', 'TMd-DMH Foxd2 Gaba'),
 ('TMv-PMv Tbx3 Hist-Gaba', 'TMv-PMv Tbx3 Hist-Gaba'),
 ('TRS-BAC Sln Glut', 'TRS-BAC Sln Glut'),
 ('TU-ARH Otp Six6 Gaba', 'TU-ARH Otp Six6 Gaba'),
 ('Tanycyte NN', 'Tanycyte NN'),
 ('VLMC NN', 'VLMC NN'),
 ('VMH Nr5a1 Glut', 'VMH Nr5a1 Glut'),
 ('Vip Gaba', 'Vip Gaba'),
 ('ZI Pax6 Gaba', 'ZI Pax6 Gaba')}

In [ ]:
pair_set =set([tuple(sorted([ct1, ct2])) for ct1, ct2 in 
     combined_results[['cell_type1', 'cell_type2']].values])
len(pair_set)

In [ ]:
len(pair_set - pair_set_15um)

In [ ]:
len(set([tuple(sorted([ct1, ct2])) for ct1, ct2 in 
     combined_results[['cell_type1', 'cell_type2']].values if ct1==ct2]))